### Converting between instruments using a Seq2Seq Model

In [1]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import librosa
import numpy as np
import random

from pathlib import Path
from torch.utils.data import Dataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Torchaudio processing methods

In [28]:
def normalized(tensor):
    # https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html
    centered = tensor - tensor.mean()
    normalized = tensor / tensor.abs().max()
    return normalized

_mu_encoder = torchaudio.transforms.MuLawEncoding()
_mu_decoder = torchaudio.transforms.MuLawDecoding()

def mu_law_encode(waveform):
    return _mu_encoder(normalized(waveform))

def mu_law_decode(waveform):
    return _mu_decoder(normalized(waveform))

def load_audio(path):
    """ Load .wav file to Mu law encoding tensor """
    waveform, sample_rate = torchaudio.load(path)
    return mu_law_encode(waveform), sample_rate

def save_audio(path, data, sample_rate):
    """ Save Mu law encoding tensor to .wav file """
    waveform = mu_law_decode(data)
    torchaudio.save(path, data, sample_rate)
    
SOS_token = 0
EOS_token = -2

Converting wav files to tensors and saving them

In [29]:
dataset = Path.cwd().joinpath("dataset")
train_wav = dataset.joinpath("train_wav")
train_tensors = dataset.joinpath("train_tensors")
test_wav = dataset.joinpath("test_wav")
test_tensors = dataset.joinpath("test_tensors")

for instrument in train_wav.iterdir():
    if (instrument.name != ".DS_Store"):
        for note in train_wav.joinpath(instrument.name).iterdir():
            if (note.name != ".DS_Store"):
                wav_tensor = load_audio(train_wav.joinpath(instrument.name, note.name))
                wav_tensor = (wav_tensor[0][0] + wav_tensor[0][1]) / 2
                torch.save(wav_tensor, train_tensors.joinpath(instrument.name, note.stem + ".pt"))
                
for instrument in test_wav.iterdir():
    if (instrument.name != ".DS_Store"):
        for note in test_wav.joinpath(instrument.name).iterdir():
            if (note.name != ".DS_Store"):
                wav_tensor = load_audio(test_wav.joinpath(instrument.name, note.name))
                wav_tensor = (wav_tensor[0][0] + wav_tensor[0][1]) / 2
                torch.save(wav_tensor, test_tensors.joinpath(instrument.name, note.stem + ".pt"))

Loading in the tensors and preparing them for the dataset

In [30]:
dataset = Path.cwd().joinpath("dataset")

#Change for model you are training
train_input = dataset.joinpath("train_tensors", "75")
train_output = dataset.joinpath("train_tensors", "81")
test_input = dataset.joinpath("test_tensors", "75")
test_output = dataset.joinpath("test_tensors", "81")

train_input_wav = []
train_output_wav = []
test_input_wav = []
test_output_wav = []

#Take out DS_Stores if you are using a Mac
for tensor in train_input.iterdir():
    if tensor.name != ".DS_Store":
        train_input_wav.append(tensor)
    
for tensor in train_output.iterdir():
    if tensor.name != ".DS_Store":
        train_output_wav.append(tensor)

for tensor in test_input.iterdir():
    if tensor.name != ".DS_Store":
        test_input_wav.append(tensor)
    
for tensor in test_output.iterdir():
    if tensor.name != ".DS_Store":
        test_output_wav.append(tensor)
        
print("train lengths:", len(train_input_wav), len(train_output_wav))
print("test lengths:", len(test_input_wav), len(test_output_wav))

train lengths: 45 45
test lengths: 15 15


In [40]:
class Seq2SeqDataset(Dataset):
    """
    Seq2Seq Dataset. Uses torchaudio + mu law to process wav files.
    """

    def __init__(self, input_wavs, output_wavs, transform=None):
        """
        Args:
            input_wavs: list of input wavs from 1st instrument
            output_wavs: list of output wavs from 2nd instrument
        """
        self.input_wavs = input_wavs
        self.output_wavs = output_wavs
        
    def __len__(self):
        return len(self.input_wavs)

    def __getitem__(self, index):
        input = torch.load(self.input_wavs[index])
        output = torch.load(self.output_wavs[index])
        return input[::2000], output[::2000]
#         return torch.cat((input[::10], torch.tensor([EOS_token])), 0), torch.cat((output[::10], torch.tensor([EOS_token])), 0)

Create data loaders using Seq2SeqDataset

In [49]:
train_set = Seq2SeqDataset(train_input_wav, train_output_wav)
test_set = Seq2SeqDataset(test_input_wav, test_output_wav)
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True, **kwargs)

seq_length = 12

Train set size: 45
Test set size: 15


In [50]:
for data, target in train_loader:
    print(data[0].size())

torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])
torch.Size([12])


In [51]:
class EncoderRNN(nn.Module):
    def __init__(self):
        super(EncoderRNN, self).__init__()
        self.gru = nn.GRU(1, 1, num_layers = 1)

    def forward(self, input, hidden):
        output, hidden = self.gru(input, hidden)
        return output, hidden

In [52]:
class DecoderRNN(nn.Module):
    def __init__(self):
        super(DecoderRNN, self).__init__()
        self.gru = nn.GRU(1, 1, num_layers = 1)

    def forward(self, input, hidden):
        output, hidden = self.gru(input, hidden)
        return output, hidden

In [53]:
encoder = EncoderRNN().to(device)
decoder = DecoderRNN().to(device)
encoder_optimizer = optim.Adam(encoder.parameters(), lr = 0.01, weight_decay = 0.0001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr = 0.01, weight_decay = 0.0001)
encoder_scheduler = optim.lr_scheduler.StepLR(encoder_optimizer, step_size = 20, gamma = 0.1)
decoder_scheduler = optim.lr_scheduler.StepLR(decoder_optimizer, step_size = 20, gamma = 0.1)
criterion = nn.MSELoss()
teacher_forcing_ratio = 0.5

In [54]:
def train(encoder, decoder, epoch):
    encoder.train()
    decoder.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        loss = 0
        
        data = torch.unsqueeze(data, 2)
        target = torch.unsqueeze(target, 2)
        data = data.permute(1, 0, 2).float()
        target = target.permute(1, 0, 2).float()
        data_len = data.size(0)
        target_len = target.size(0)
        batch_len = data.size(1)
        
        encoder_hidden = torch.zeros([1, batch_len, 1], dtype=torch.float, device=device)
        encoder_output, encoder_hidden = encoder(data, encoder_hidden)

        decoder_input = torch.zeros([1, batch_len, 1], dtype=torch.float, device=device)
        for i in range(target.size(1)):
            decoder_input[0][i] = torch.tensor([[SOS_token]], dtype=torch.float, device=device)
        
        decoder_hidden = encoder_hidden
        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
        
        if use_teacher_forcing:
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output.view(-1, 1), target[0])
            for i in range(target_len - 1):
                decoder_output, decoder_hidden = decoder(target[i].unsqueeze(0), decoder_hidden)
                loss += criterion(decoder_output.view(-1, 1), target[i + 1])
        else:
            for i in range(target_len):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                loss += criterion(decoder_output.view(-1, 1), target[i])
                decoder_input = decoder_output
        
#         loss /= seq_length
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} Batch: {}\tLoss: {:.6f}'.format(
                epoch, batch_idx, loss))

In [55]:
def test(encoder, decoder, epoch):
    encoder.eval()
    decoder.eval()
    for data, target in test_loader:
        loss = 0
        
        data = torch.unsqueeze(data, 2)
        target = torch.unsqueeze(target, 2)
        data = data.permute(1, 0, 2).float()
        target = target.permute(1, 0, 2).float()
        data_len = data.size(0)
        target_len = target.size(0)
        batch_len = data.size(1)
        
        encoder_hidden = torch.zeros([1, batch_len, 1], dtype=torch.float, device=device)
        encoder_output, encoder_hidden = encoder(data, encoder_hidden)

        decoder_input = torch.zeros([1, batch_len, 1], dtype=torch.float, device=device)
        for i in range(target.size(1)):
            decoder_input[0][i] = torch.tensor([[SOS_token]], dtype=torch.float, device=device)
        
        decoder_hidden = encoder_hidden
        
        for i in range(target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output.view(-1, 1), target[i])
            decoder_input = decoder_output
        
#         loss /= seq_length
    print('\nTest set: Loss: {}\n'.format(loss))

In [56]:
log_interval = 2
for epoch in range(1, 41):
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
    encoder_scheduler.step()
    decoder_scheduler.step()
    train(encoder, decoder, epoch)
    test(encoder, decoder, epoch)

Train Epoch: 1 Batch: 0	Loss: 247325.062500
Train Epoch: 1 Batch: 2	Loss: 244293.078125
Train Epoch: 1 Batch: 4	Loss: 217919.953125

Test set: Loss: 234219.234375

Train Epoch: 2 Batch: 0	Loss: 218389.468750
Train Epoch: 2 Batch: 2	Loss: 240846.437500
Train Epoch: 2 Batch: 4	Loss: 235160.812500

Test set: Loss: 232446.125

Train Epoch: 3 Batch: 0	Loss: 241247.468750
Train Epoch: 3 Batch: 2	Loss: 236456.515625
Train Epoch: 3 Batch: 4	Loss: 246191.281250

Test set: Loss: 239718.015625

Train Epoch: 4 Batch: 0	Loss: 248954.828125
Train Epoch: 4 Batch: 2	Loss: 222878.421875
Train Epoch: 4 Batch: 4	Loss: 243919.171875

Test set: Loss: 229102.046875

Train Epoch: 5 Batch: 0	Loss: 221610.109375
Train Epoch: 5 Batch: 2	Loss: 211299.468750
Train Epoch: 5 Batch: 4	Loss: 264082.125000

Test set: Loss: 237163.6875

Train Epoch: 6 Batch: 0	Loss: 208614.234375
Train Epoch: 6 Batch: 2	Loss: 223187.531250
Train Epoch: 6 Batch: 4	Loss: 248786.750000

Test set: Loss: 238508.0

Train Epoch: 7 Batch: 0	Lo